In [53]:
# Note: https://ec.europa.eu/jrc/en/publication/global-hourly-solar-radiation-data-set-using-satellite-and-reanalysis-data

def solar_angle(lat,lon,localtime,timezone,day_of_year):
    '''
    Calculate solar angle 
    Source: https://www.esrl.noaa.gov/gmd/grad/solcalc/solareqns.PDF, returns in rad
    '''
    hour = localtime.hour
    minute = localtime.minute
    
    # Fractional year in radians
    frac_year = 2*math.pi/365*(day_of_year - 1 + (hour - 12)/24)

    # Equation of time in minutes
    eqtime = 229.18*(0.000075 + 0.001868*math.cos(frac_year)-
                     0.032077*math.sin(frac_year)-0.014615*math.cos(2*frac_year)-
                     0.040849*math.sin(2*frac_year))

    # Declination angle in radians
    decl = 0.006918 - 0.399912*math.cos(frac_year) \
            + 0.070257*math.sin(frac_year) - 0.006758*math.cos(2*frac_year) \
            + 0.000907*math.sin(2*frac_year)-0.002697*math.cos(3*frac_year) \
            + 0.00148*math.sin(3*frac_year)

    # Time offset in minutes 
    time_offset = eqtime + 4*lon-60*timezone
    
    tst = hour*60 + minute + time_offset

    # Solar hour angle in degrees
    ha = (tst/4) - 180    

    elev_angle_rad = math.asin(math.sin(math.pi/180*lat) * math.sin(decl) + math.cos(math.pi/180*lat)*
                    math.cos(decl)*math.cos(math.pi/180*ha))

    return max(elev_angle_rad,0)


def get_illuminance(ghi,dhi,dni,zenith_deg,rel_airmass,dewPoint):
    '''
    Source: Generate illuminance values per 'Modeling Daylight Availability and Irradiance Components from
    Direct and Global Irradiance', Perez R. (1990)
    
    glbeIll, drctIll, diffIll, zenIll = get_illuminance(ghi,dhi,dni,zenith_deg,rel_airmass,dewPoint)
    
    Note: Currently uses actual zenith rather than apparent zenith
    '''
    
    zenith = zenith_deg *math.pi/180
    
    if zenith < math.pi/2-(0.5*math.pi/180.) and ghi > 0:
        kai = 1.041
        eps = ((dhi+dni)/dhi + kai*zenith**3)/(1+kai*zenith**3)
        delta = dhi*rel_airmass/1360
        W = math.exp(0.08*dewPoint-0.075)

        # Per Perez Table 1: Discrete Sky Clearness Categories
        if eps >= 1 and eps < 1.065:
            e_category = 1
        elif eps >= 1.065 and eps < 1.23:
            e_category = 2
        elif eps >= 1.23 and eps < 1.5:
            e_category = 3
        elif eps >= 1.5 and eps < 1.95:
            e_category = 4
        elif eps >= 1.95 and eps < 2.8:
            e_category = 5
        elif eps >= 2.8 and eps < 4.5:
            e_category = 6
        elif eps >= 4.5 and eps < 6.2:
            e_category = 7
        elif eps >= 6:
            e_category = 8
        else:
            print('error')
            return None

        # Per Perez Table 4: Luminous Efficacy
        GlobLumEff = {}
        GlobLumEff[1] = ( 96.63, -0.47, 11.50,  -9.16)
        GlobLumEff[2] = (107.54,  0.79,  1.79,  -1.19)
        GlobLumEff[3] = ( 98.73,  0.70,  4.40,  -6.95)
        GlobLumEff[4] = ( 92.72,  0.56,  8.36,  -8.31)
        GlobLumEff[5] = ( 86.73,  0.98,  7.10, -10.94)
        GlobLumEff[6] = ( 88.34,  1.39,  6.06,  -7.60)
        GlobLumEff[7] = ( 78.63,  1.47,  4.93, -11.37)
        GlobLumEff[8] = ( 99.65,  1.86, -4.46,  -3.15)

        # Eq 6
        a,b,c,d = GlobLumEff[e_category]
        glbeIll = int(ghi*(a+b*W+c*math.cos(zenith)+d*math.log(delta)))

        DirLumEff = {}
        DirLumEff[1] = ( 57.20, -4.55,  -2.98, 117.12)
        DirLumEff[2] = ( 98.99, -3.46,  -1.21,  12.38)
        DirLumEff[3] = (109.83, -4.90,  -1.71,  -8.81)
        DirLumEff[4] = (110.34, -5.84,  -1.99,  -4.56)
        DirLumEff[5] = (106.36, -3.97,  -1.75,  -6.16)
        DirLumEff[6] = (107.19, -1.25,  -1.51, -26.73)
        DirLumEff[7] = (105.75,  0.77,  -1.26, -34.44)
        DirLumEff[8] = (101.18,  1.58,  -1.10,  -8.29)

        # Eq 8
        a,b,c,d = DirLumEff[e_category]
        drctIll = int(max(0,dni*(a+b*W+c*math.exp(5.73*zenith-5)+d*delta)))

        DiffLumEff = {}
        DiffLumEff[1] = ( 97.24, -0.46,  12.00,  -8.91)
        DiffLumEff[2] = (107.22,  1.15,   0.59,  -3.95)
        DiffLumEff[3] = (104.97,  2.96,  -5.52,  -8.77)
        DiffLumEff[4] = (102.39,  5.59, -13.95, -13.90)
        DiffLumEff[5] = (100.71,  5.94, -22.75, -23.74)
        DiffLumEff[6] = (106.42,  3.83, -36.15, -28.83)
        DiffLumEff[7] = (141.88,  1.90, -53.24, -14.03)
        DiffLumEff[8] = (152.23,  0.35, -45.27,  -7.98)

        # Eq 7
        a,b,c,d = DiffLumEff[e_category]
        diffIll = int(dhi*(a + b*W + c*math.cos(zenith) + d*math.log(delta)))

        ZenLumEff = {}
        ZenLumEff[1] = ( 40.86, 26.77, -29.59, -45.75)
        ZenLumEff[2] = ( 26.58, 14.73,  58.46, -21.25)
        ZenLumEff[3] = ( 19.34,  2.28, 100.00,   0.25)
        ZenLumEff[4] = ( 13.25, -1.39, 124.79,  15.66)
        ZenLumEff[5] = ( 14.47, -5.09, 160.09,   9.13)
        ZenLumEff[6] = ( 19.76, -3.88, 154.61, -19.21)
        ZenLumEff[7] = ( 28.39, -9.67, 151.58, -69.39)
        ZenLumEff[8] = ( 42.91,-19.62, 130.80,-164.08)

        # Eq 9
        a,b,c,d = ZenLumEff[e_category]
        zenIll = int(dhi*(a+b*math.cos(zenith)+c*math.exp(-3*zenith)+d*delta))
        
    # If sun is below horizon, return 0
    else:
        glbeIll, drctIll, diffIll, zenIll = 0, 0, 0, 0
    
    return glbeIll, drctIll, diffIll, zenIll


def extNorRad(dayofyear):
    '''
    ref: pg. 32 of 'SolarRadiationCalculation.pdf' (saved on Desktop)
    '''

    Gsc = 1367.0
    return Gsc*(1+0.033*math.cos((360*dayofyear/365*np.pi/180)))


def extHorRad(lat,lon,timezone,time):
    zenith = np.pi/2 - solar_angle(lat,lon,time,timezone,45)
    
    return round(extNorRad(time.timetuple().tm_yday)*math.cos(zenith),0)


def AshraeClearSky(beta,Eo,tau_b,tau_d):
    '''
    ASHRAE 2009 Clear Sky Model
    '''
    m = 1/(math.sin(beta) + 0.50572*(6.07995+beta*180/np.pi)**(-1.6364))

    ab = 1.219 - 0.043 * tau_b - 0.151 * tau_d - 0.204 * tau_b * tau_d
    ad = 0.202 + 0.852 * tau_b - 0.007 * tau_d - 0.357 * tau_b * tau_d
    
    Eb = Eo * math.exp(-tau_b*m**(ab))
    Ed = Eo * math.exp(-tau_d*m**(ad))
    
    return Eb,Ed


def get_weather(year,lat,lon):
    import xarray as xr
    import os
    import pandas as pd
    
    directory = str(year) + 'c'
    
    parameters = ['2m_dewpoint_temperature',
           't2m',
#           '10m_wind_direction',    # These are stored differently (0.5 x 0.5, and every 2 hours only - need to fix)
#           '10m_wind_speed',
           'cloud_base_height',
           'snow_depth',
           'soil_temperature_level_4',
           'surface_pressure',
           'surface_solar_radiation_downwards',
           'surface_thermal_radiation_downwards',
           'total_cloud_cover',
           'total_precipitation',
           'total_sky_direct_solar_radiation_at_surface'
          ]
    
    # Dataframe to return extracted parameter values
    df = pd.DataFrame()
    
    for month in range(1,13):
        data = []
        
        for param in parameters:
            ifile = '%dc//%s_%d_%02d_era5.nc'%(year,param,year,month)

            if os.path.isfile(ifile) and os.path.getsize(ifile) > 1e8:
                ds = xr.open_dataset(ifile)
                da = ds.sel(latitude=lat,longitude=lon%360,method='nearest')
                data.append(da)

        df = df.append(xr.merge(data).to_dataframe())
        
    df['ssrd'] = (df['ssrd']/3600).astype(int)     # Convert from J/m^2 to W/m^2
    df['strd'] = (df['strd']/3600).astype(int)           
    df['fdir'] = (df['fdir']/3600).astype(int)
    df['d2m'] = (df['d2m'] - 273.15).round(2)      # Convert from K to C
    df['t2m'] = (df['t2m'] - 273.15).round(2)      # Convert from K to C
    df['stl4'] = (df['stl4'] - 273.15).round(2)    # Convert from K to C
    df['sp'] = df['sp'].astype(int)
    df['cbh'] = df['cbh'].fillna(-1).astype(int)              # Round to whole numbers (cloud base height)
    
    df = df.drop(['latitude','longitude'],axis=1)
    
    return df

In [ ]:
year = 2010
lat = 42.36
lon = -71.0

%prun -l 20 weather = get_weather(year,lat,lon)

In [52]:
weather

,dwi,wind
time,,
2011-01-01 00:00:00,NaN,NaN
2011-01-01 01:00:00,NaN,NaN
2011-01-01 02:00:00,NaN,NaN
2011-01-01 03:00:00,NaN,NaN
2011-01-01 04:00:00,NaN,NaN
2011-01-01 05:00:00,NaN,NaN
2011-01-01 06:00:00,NaN,NaN
2011-01-01 07:00:00,NaN,NaN
2011-01-01 08:00:00,NaN,NaN
